### OPEN API DATA_TO CSV

In [ ]:
import requests
import json
import csv

# CSV 파일 저장 경로 설정
##### 저장할 CSV파일의 경로를 이곳에 설정하세요 ##### 
csv_file_path = "C:/Users/user/Desktop/Public_data/2024_충청북도_공공데이터_창업경진대회/행정안전부_정책연구_과제정보.csv"


# API 요청 기본 정보
### API url과 필요한 변수명을 List로 만드세요 ###
### 파라미터의 변수의 종류는 메타데이터를 참고하세요 ###
url = '	http://apis.data.go.kr/1741000/prism_v2'
type = 'json'
api_types = ['youApi', 'equApi', 'famApi', 'proApi', 'othApi']  # 사용할 apiType 목록
### param2 = []
### param3 = []

### encoding = '발급받은 인코딩 인증키를 복사하여 붙여넣기 해 주세요.'
decoding = 'mW18J6e5gQ61JGt5YYgjiPhATkpOt/KJJxNjPUP3j/RoQF4VCrtduhvdMHs52rOOgdKh15FqXmnsPSsk8avWWA=='


# 최초의 API 호출로부터 칼럼명 동적 생성
def fetch_initial_columns(url, params):
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        json_data = response.json()
        first_item = json_data['body'][0]['items']['item'][0]  # 첫 번째 아이템 접근
        return list(first_item.keys())  # 모든 키를 칼럼명으로 사용
    except (requests.RequestException, ValueError) as e:
        print(f"초기 칼럼명 추출 실패: {e}")
        return []

# CSV 파일 오픈 및 헤더 작성
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 최초 API 호출을 통해 칼럼명 설정
    
    ##############################################
    ####파라미터를 데이터에 알맞게 설정하세요#####
    ##############################################
    
    initial_params = {
        'serviceKey' : decoding, 
        'type' : type,
        'numOFRows' : '10',
        'pageNo' : '1',
        'apiType' : api_types[0]  # 첫 번째 API 타입으로 칼럼명 결정
    }
    columns = fetch_initial_columns(url, initial_params)
    writer.writerow(columns)
    
    # 각 API 타입과 페이지에 대해 반복
    for api_type in api_types:
        
    ##############################################
    ####파라미터를 데이터에 알맞게 설정하세요#####
    ##############################################
        
        params = {
            'serviceKey' : decoding,
            'type' : type,
            'numOFRows' : '10',
            'apiType' : api_type
        }
        for page in range(1, 100):
            params['pageNo'] = page
            try:
                response = requests.get(url, params=params)
                response.raise_for_status()  # 요청 실패 시 예외 발생
                json_data = response.json()
                items = json_data['body'][0]['items']['item']
        
                # 각 항목별 데이터를 CSV 파일에 추가
                for item in items:
                    row = [item.get(col, '') for col in columns]  # 동적 칼럼명을 사용하여 데이터 추출
                    writer.writerow(row)
            except (requests.RequestException, ValueError) as e:
                print(f"페이지 {page} 처리 중 오류 발생: {e}")

print(f"CSV 파일이 저장된 위치: {csv_file_path}")